<a href="https://colab.research.google.com/github/YusolCho/Basic/blob/main/LoL_Match_majority_divison1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **RANK = GOLD, TIER(DIVISION) = I 인 유저들의 매치 데이터셋 생성하기**
## **(유저 20000명으로 확대)**

* MIDDLE 로 포지션을 제한시켜버리면 1/8정도로 데이터가 확 감소하기 때문에, 초반에 많은 양의 데이터셋을 만들고자 함 

##  ** << 만드는 순서 >> ** 

###  1. **SUMMONER ID, SUMMONER NAME 추출** (LEAGUE-EXP-V4: region 설정 'KR'로 해야함)
    * 데이터 프레임 타입으로 추출 
    * SUMMONER ID 만 따로 리스트로 추출하고 2번으로 

###  2. **PUUID 추출** (SUMMONER-V4: region 설정 'KR'로 해야함)
    * 원소가 문자열인 리스트 타입으로 추출

###  3. **MATCH ID 추출** (MATCH-V5: region 설정 'asia'로 해야함)
    * 하나의 PUUID 당 몇 개의 MATCH ID 를 추출한건지를 생각해야함. 
    * PUUID의 개수(SUMMONER NAME, SUMMONER ID 개수와 동일, len(puuid 리스트명)으로 확인)가 충분히 많다면 PUUID 하나당 MATCH ID 하나만 뽑아도 괜찮음 
    * 이때, 반드시 PUUID와 그에 따른 MATCH ID 가 세트로 저장되게 해야함.
      * ex1) PUUID 를 key값, 그에 따른  MATCH ID를 value값으로 하는 딕셔너리 타입으로 만들기
      * ex2) PUUID와 그에 따른 MATCH ID를 동일한 행에 넣는 데이터 프레임 타입으로 만들기 
    * 즉, 어떤 PUUID 로부터 각각의 MATCH-ID 가 추출되었는지가 결과에서 드러나도록 해야함 
    * 그래야 4번 단계에서 문제가 안생김

###  4. **MATCH DATA 추출** (MATCH-V5: region 설정 'asia'로 해야함)
    * 이 단계에서 시간이 줜나 오래걸림
    * 하나의 match id 에 대해서 match data를 뽑을 때, 그 match data 안에는 총 10명정도의 참여자에 대한 게임 활동 정보가 들어있음. 
    * 우리가 원하는 건 그 match id 를 뽑을 때 사용한 puuid에 해당되는 유저의 개임 활동 정보임.
    * 따라서 그 puuid에 해당되는 participant 의 데이터만 뽑아줘야 함.
    * 뽑아야하는 변수들(리스트꼴로 만든 뒤 데이터프레임의 각 열로 만들면 됨) 
     1. game_mode_list = [] 얘넨 나중에
     2. game_type_list = []
     3. assists_list = []
     4. kills_list = []
     5. deaths_list = []
     6. champExp_list = [] #champ_experience_list
     7. champName_list = [] #champion_name_list
     8. indiPos_list = [] #individual_position_list
     9. lane_list = []
     10. role_list = []
     11. gold_list = []   #gold_earned_list
     12. teamPos_list = []
     13. tDDealt_list = []  #total_damage_dealt_list
     14. tDDealtChamp_list = [] #total_damage_dealt_to_champions_list
     15. tDTaken_list = []  #total_damage_taken_list
     16. tMKilled_list = []  #total_minions_killed_list
     17. turKills_list = []  #turret_kills_list
     18. win_list = []



### 환경설정

In [7]:
#API KEY 미리 지정하고 시작 
api_key = 'RGAPI-b1697406-1a4c-4ed5-85b8-f2e043da9aec'
# SET UP 
base_url = 'https://{region}.api.riotgames.com'

In [8]:
# 필요한 패키지 불러오기 
import requests
import pandas as pd
import time
import json

# **1.GOLD, DIVISION I 에 해당되는 유저들의 SUMMONER ID, SUMMONER NAME 데이터프레임으로 뽑기**

In [ ]:
# SET UP 
base_url = 'https://{region}.api.riotgames.com'
regions = ['kr']
rank = 'GOLD'
division = 'I'
user_data_1 = []

# Iterate over each region
for region in regions:
    # Iterate over each page of user data
    for page in range(1, 99):
        # Set up the League API endpoint URL
        league_url = base_url.format(region=region) + '/lol/league-exp/v4/entries/RANKED_SOLO_5x5/{tier}/{division}?page={page}&api_key={api_key}'
        # Set up the headers for the API requests
        headers = {'X-Riot-Token': api_key}
        
        # Make a request
        response = requests.get(league_url.format(tier=rank, division=division, page=page, api_key=api_key), headers=headers)
        league_data = response.json()

        # If there is no more data, break out
        if not league_data:
            break
    
        # Iterate over each user and collect their data
        for user in league_data:
            user_dict_1 = {
                'summoner_id' : user['summonerId'],
                'summoner_name': user['summonerName']
            }
            user_data_1.append(user_dict_1)
        
        # Delay for 2 seconds between API requests to avoid rate limit errors
        time.sleep(1) ############ 2에서 1로 바꿈

# List to DataFrame
smmr_id_name_df = pd.DataFrame(user_data_1)

# CHECK
print(smmr_id_name_df)


#### 여기서 summoner_id 변수를 list 타입으로 따로 저장한다(summoner_id_list). 



In [ ]:
summoner_id_list = smmr_id_name_df['summoner_id'].tolist()
print(summoner_id_list)

In [ ]:
# csv 파일로 저장
import csv
f = open('summoner_id_list_divisionI.csv', 'w')
writer = csv.writer(f)
writer.writerow(summoner_id_list)
f.close()

In [ ]:
# csv 파일을 리스트로 불러오기 
import csv 
summoner_id_list = []

with open('/content/drive/MyDrive/LOL_ANALYSIS_0323/summoner_id_list_divisionI.csv','r',newline='') as f:
    reader = csv.reader(f)
    for row in reader:
        summoner_id_list.append(row)

print(summoner_id_list)

In [ ]:
# 중복 괄호 제거용 코드
a = []  
a.extend(summoner_id_list[0])  
summoner_id_list = a
print(summoner_id_list)

# **2. PUUID 리스트 타입으로 추출**

In [ ]:
pip install retrying  # 필요한 패키지 설치 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import retrying 

# **너무 느린 관계로.. 코랩 창 10개 열고 나눠서 돌리기로 한다.**
### **ID1**

In [ ]:
smmrid_1 = summoner_id_list[0:200]
smmrid_2 = summoner_id_list[200:400]
smmrid_3 = summoner_id_list[400:600]
smmrid_4 = summoner_id_list[600:800]
smmrid_5 = summoner_id_list[800:1000]
smmrid_6 = summoner_id_list[1000:1200]
smmrid_7 = summoner_id_list[1200:1400]
smmrid_8 = summoner_id_list[1400:1600]
smmrid_9 = summoner_id_list[1600:1800]
smmrid_10 = summoner_id_list[1800:2000]

In [ ]:
#ID1
# Set up the regions 
regions = ['kr']

# Set up an empty list to store the puuids
puuid_list = []

# Set up rate limit variables
RATE_LIMIT_RETRIES = 5
RATE_LIMIT_BACKOFF_FACTOR = 2

# Set up retries variables
RETRIES = 5
BACKOFF_FACTOR = 2

# Define a function to make a request to the Riot API with retries and rate limiting
@retrying.retry(wait_exponential_multiplier=1000, wait_exponential_max=10000, stop_max_attempt_number=RETRIES)
def request_riot_api(url, headers):
    response = requests.get(url, headers=headers)
    if response.status_code == 429:
        retry_after = int(response.headers['Retry-After'])
        time.sleep(retry_after + 0.1)
        raise Exception('Rate limit exceeded, retrying...')
    elif response.status_code != 200:
        raise Exception(f'Request failed with status code {response.status_code}: {response.content.decode()}')
    return response.json()

# Iterate over each region
for region in regions:
    # Set up the Summoner API endpoint URL
    summoner_url = base_url.format(region=region) + '/lol/summoner/v4/summoners/{encryptedSummonerId}'
    
    # Set up the headers for the API requests
    headers = {'X-Riot-Token': api_key}
    
    # Iterate over each summoner ID in the list
    for summoner_id in smmrid_1:########################
        # Make a request to the Summoner API for the corresponding puuid
        for i in range(RATE_LIMIT_RETRIES):
            try:
                summoner_data = request_riot_api(summoner_url.format(encryptedSummonerId=summoner_id), headers)
                break
            except Exception as e:
                print(e)
                print(f'Retrying in {BACKOFF_FACTOR ** i} seconds...')
                time.sleep(BACKOFF_FACTOR ** i)

        # Append the puuid to the list
        puuid_list.append(summoner_data['puuid'])

# Print the resulting puuid list
print(puuid_list)

In [ ]:
f = open('PUUID1(0326).csv', 'w')
writer = csv.writer(f)
writer.writerow(puuid_list)
f.close()

# **PUUID 를 하나의 리스트로 합치기**


In [ ]:
# csv 파일을 리스트로 불러오기 
import csv 

puuid_list_1 = []

with open('/content/drive/MyDrive/LOL_ANALYSIS_0323/PUUID1(0326).csv','r',newline='') as f:
    reader = csv.reader(f)
    for row in reader:
        puuid_list_1.append(row)

print(puuid_list_1)


# 중복 괄호 제거용 코드
a1 = []  
a1.extend(puuid_list_1[0])  
puuid_list_1 = a1
print(puuid_list_1)

In [ ]:
# csv 파일을 리스트로 불러오기 
import csv 

puuid_list_2 = []

with open('/content/drive/MyDrive/LOL_ANALYSIS_0323/PUUID2(0326).csv','r',newline='') as f:
    reader = csv.reader(f)
    for row in reader:
        puuid_list_2.append(row)

print(puuid_list_2


# 중복 괄호 제거용 코드
a2 = []
a2.extend(puuid_list_2[0])  
puuid_list_2 = a2
print(puuid_list_2)

In [ ]:
# csv 파일을 리스트로 불러오기 
import csv 

puuid_list_3 = []

with open('/content/drive/MyDrive/LOL_ANALYSIS_0323/PUUID3(0326).csv','r',newline='') as f:
    reader = csv.reader(f)
    for row in reader:
        puuid_list_3.append(row)

print(puuid_list_3)


# 중복 괄호 제거용 코드
a3 = [] 
a3.extend(puuid_list_3[0])  
puuid_list_3 = a3
print(puuid_list_3)

In [ ]:
# csv 파일을 리스트로 불러오기 
import csv 

puuid_list_4 = []

with open('/content/drive/MyDrive/LOL_ANALYSIS_0323/PUUID4(0326).csv','r',newline='') as f:
    reader = csv.reader(f)
    for row in reader:
        puuid_list_4.append(row)

print(puuid_list_4)


# 중복 괄호 제거용 코드
a4 = []  
a4.extend(puuid_list_4[0])  
puuid_list_4 = a4
print(puuid_list_4)

In [ ]:
# csv 파일을 리스트로 불러오기 
import csv 

puuid_list_5 = []

with open('/content/drive/MyDrive/LOL_ANALYSIS_0323/PUUID5(0326).csv','r',newline='') as f:
    reader = csv.reader(f)
    for row in reader:
        puuid_list_5.append(row)

print(puuid_list_5)


# 중복 괄호 제거용 코드
a5 = []  
a5.extend(puuid_list_5[0])  
puuid_list_5 = a5
print(puuid_list_5)

In [ ]:
# csv 파일을 리스트로 불러오기 
import csv 
puuid_list_6 = []
with open('/content/drive/MyDrive/LOL_ANALYSIS_0323/PUUID6(0326).csv','r',newline='') as f:
    reader = csv.reader(f)
    for row in reader:
        puuid_list_6.append(row0
print(puuid_list_6)
# 중복 괄호 제거용 코드
a6 = []   
a6.extend(puuid_list_6[0])   
puuid_list_6 = a6 
print(puuid_list_6)

In [ ]:
# csv 파일을 리스트로 불러오기 
import csv 
puuid_list_7 = []
with open('/content/drive/MyDrive/LOL_ANALYSIS_0323/PUUID7(0326).csv','r',newline='') as f:
    reader = csv.reader(f)
    for row in reader:
        puuid_list_7.append(row)
print(puuid_list_7)
# 중복 괄호 제거용 코드
a7 = []  
a7.extend(puuid_list_7[0])  
puuid_list_7 = a7
print(puuid_list_7)

In [ ]:
# csv 파일을 리스트로 불러오기 
import csv 
puuid_list_8 = []
with open('/content/drive/MyDrive/LOL_ANALYSIS_0323/PUUID8(0326).csv','r',newline='') as f:
    reader = csv.reader(f)
    for row in reader:
        puuid_list_8.append(row)
print(puuid_list_8)
# 중복 괄호 제거용 코드
a8 = []  
a8.extend(puuid_list_8[0])  
puuid_list_8 = a8
print(puuid_list_80

In [ ]:
# csv 파일을 리스트로 불러오기 
import csv 
puuid_list_9 = []
with open('/content/drive/MyDrive/LOL_ANALYSIS_0323/PUUID9(0326).csv','r',newline='') as f:
    reader = csv.reader(f)
    for row in reader:
        puuid_list_9.append(row)
print(puuid_list_9)
# 중복 괄호 제거용 코드
a9 = []  
a9.extend(puuid_list_9[0])  
puuid_list_9 = a9
print(puuid_list_9)

In [ ]:
# csv 파일을 리스트로 불러오기 
import csv 
puuid_list_10 = []
with open('/content/drive/MyDrive/LOL_ANALYSIS_0323/PUUID10(0326).csv','r',newline='') as f:
    reader = csv.reader(f)
    for row in reader:
        puuid_list_10.append(row)
print(puuid_list_10)
# 중복 괄호 제거용 코드
a10 = []  
a10.extend(puuid_list_10[0])  
puuid_list_10 = a10
print(puuid_list_10)

In [ ]:
puuid_list =  puuid_list_1+ puuid_list_2 + puuid_list_3 + puuid_list_4 + puuid_list_5 + puuid_list_6 + puuid_list_7 + puuid_list_8 + puuid_list_9 + puuid_list_10  
print(len(puuid_list))

2000


In [ ]:
# puuid list 를 csv 파일로 저장하기
import csv
f = open('puuid(0326).csv', 'w')
writer = csv.writer(f)
writer.writerow(puuid_list)
f.close()

In [ ]:
# csv 파일을 리스트로 불러오기 
import csv 
puuid_list = []

with open('/content/drive/MyDrive/LOL_ANALYSIS_0323/puuid(0326).csv','r',newline='') as f:
    reader = csv.reader(f)
    for row in reader:
        puuid_list.append(row)

print(puuid_list)


# 중복 괄호 제거용 코드
a = []  
a.extend(puuid_list[0])  
puuid_list = a
print(puuid_list)
print(len(puuid_list))

# **3. MATCH ID 얻기**

In [ ]:
puuid_list_1 = puuid_list[0:200]
puuid_list_2 = puuid_list[200:400]
puuid_list_3 = puuid_list[400:600]
puuid_list_4 = puuid_list[600:800]
puuid_list_5 = puuid_list[800:1000]
puuid_list_6 = puuid_list[1000:1200]
puuid_list_7 = puuid_list[1200:1400]
puuid_list_8 = puuid_list[1400:1600]
puuid_list_9 = puuid_list[1600:1800]
puuid_list_10 = puuid_list[1800:2000]
puuid_list_mini = puuid_list[0:10]

### **match list 1 뽑고 저장**

In [ ]:
# 그냥 puuid_list 에 대해 적용

match_id_list_1 = []

# Define the maximum number of requests per second and the time to wait between each batch of requests
MAX_REQUESTS_PER_SECOND = 5
WAIT_TIME_BETWEEN_BATCHES = 1 / MAX_REQUESTS_PER_SECOND

# Loop through each puuid in the list
for puuid in puuid_list_1:
    # Get the match list for the current user
    response = requests.get(f'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=10',
                            headers={'X-Riot-Token': api_key})
    
    # Check if the response was successful
    if response.status_code != 200:
        print(f'Error getting match list for puuid {puuid}: {response.text}')
        continue
    
    # Extract up to 10 match IDs for the current user
    match_ids_1 = response.json()
    
    # Add the match IDs to the final list
    match_id_list_1.extend(match_ids_1)
    
    # Sleep for a short time to avoid hitting the rate limit
    time.sleep(2)

print(match_id_list_1)


In [ ]:
# csv 파일로 저장
import csv
f = open('matchid_1(0326).csv', 'w')
writer = csv.writer(f)
writer.writerow(match_id_list_1)
f.close()

### **match list 2 뽑고 저장**

In [ ]:
# Match id list 2

match_id_list_2 = []

# Define the maximum number of requests per second and the time to wait between each batch of requests
MAX_REQUESTS_PER_SECOND = 5
WAIT_TIME_BETWEEN_BATCHES = 1 / MAX_REQUESTS_PER_SECOND

# Loop through each puuid in the list
for puuid in puuid_list_2:
    # Get the match list for the current user
    response = requests.get(f'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=10',
                            headers={'X-Riot-Token': api_key})
    
    # Check if the response was successful
    if response.status_code != 200:
        print(f'Error getting match list for puuid {puuid}: {response.text}')
        continue
    
    # Extract up to 10 match IDs for the current user
    match_ids_2 = response.json()
    
    # Add the match IDs to the final list
    match_id_list_2.extend(match_ids_2)
    
    # Sleep for a short time to avoid hitting the rate limit
    time.sleep(2)

print(match_id_list_2)

In [ ]:
# csv 파일로 저장
import csv
f = open('matchid_2(0326).csv', 'w')
writer = csv.writer(f)
writer.writerow(match_id_list_2)
f.close()

### **match list 3 뽑고 저장**

In [ ]:
# 그냥 puuid_list 에 대해 적용
match_id_list_3 = []

# Define the maximum number of requests per second and the time to wait between each batch of requests
MAX_REQUESTS_PER_SECOND = 5
WAIT_TIME_BETWEEN_BATCHES = 1 / MAX_REQUESTS_PER_SECOND

# Loop through each puuid in the list
for puuid in puuid_list_3:
    # Get the match list for the current user
    response = requests.get(f'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=10',
                            headers={'X-Riot-Token': api_key})
    
    # Check if the response was successful
    if response.status_code != 200:
        print(f'Error getting match list for puuid {puuid}: {response.text}')
        continue
    
    # Extract up to 10 match IDs for the current user
    match_ids_3 = response.json()
    
    # Add the match IDs to the final list
    match_id_list_3.extend(match_ids_3)
    
    # Sleep for a short time to avoid hitting the rate limit
    time.sleep(2)

print(match_id_list_3)

In [ ]:
# csv 파일로 저장
import csv
f = open('matchid_3(0326).csv', 'w')
writer = csv.writer(f)
writer.writerow(match_id_list_3)
f.close()

### **match list 4 뽑고 저장**

In [ ]:
import retrying 

In [ ]:
# 그냥 puuid_list 에 대해 적용
match_id_list_4 = []

# Define the maximum number of requests per second and the time to wait between each batch of requests
MAX_REQUESTS_PER_SECOND = 5
WAIT_TIME_BETWEEN_BATCHES = 1 / MAX_REQUESTS_PER_SECOND

# Loop through each puuid in the list
for puuid in puuid_list_4:
    # Get the match list for the current user
    response = requests.get(f'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=10',
                            headers={'X-Riot-Token': api_key})
    
    # Check if the response was successful
    if response.status_code != 200:
        print(f'Error getting match list for puuid {puuid}: {response.text}')
        continue
    
    # Extract up to 10 match IDs for the current user
    match_ids_4 = response.json()
    
    # Add the match IDs to the final list
    match_id_list_4.extend(match_ids_4)
    
    # Sleep for a short time to avoid hitting the rate limit
    time.sleep(2)

print(match_id_list_4)

In [ ]:
# csv 파일로 저장
import csv
f = open('matchid_4(0326).csv', 'w')
writer = csv.writer(f)
writer.writerow(match_id_list_4)
f.close()

### **match id 5 뽑고 저장**

In [ ]:
# 그냥 puuid_list 에 대해 적용
match_id_list_5 = []

# Define the maximum number of requests per second and the time to wait between each batch of requests
MAX_REQUESTS_PER_SECOND = 5
WAIT_TIME_BETWEEN_BATCHES = 1 / MAX_REQUESTS_PER_SECOND

# Loop through each puuid in the list
for puuid in puuid_list_5:
    # Get the match list for the current user
    response = requests.get(f'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=10',
                            headers={'X-Riot-Token': api_key})
    
    # Check if the response was successful
    if response.status_code != 200:
        print(f'Error getting match list for puuid {puuid}: {response.text}')
        continue
    
    # Extract up to 10 match IDs for the current user
    match_ids_5 = response.json()
    
    # Add the match IDs to the final list
    match_id_list_5.extend(match_ids_5)
    
    # Sleep for a short time to avoid hitting the rate limit
    time.sleep(2)

print(match_id_list_5)

In [ ]:
# csv 파일로 저장
import csv
f = open('matchid_5(0326).csv', 'w')
writer = csv.writer(f)
writer.writerow(match_id_list_5)
f.close()

### **match id 6 뽑고 저장**

In [ ]:
# 그냥 puuid_list 에 대해 적용
match_id_list_6 = []

# Define the maximum number of requests per second and the time to wait between each batch of requests
MAX_REQUESTS_PER_SECOND = 5
WAIT_TIME_BETWEEN_BATCHES = 1 / MAX_REQUESTS_PER_SECOND

# Loop through each puuid in the list
for puuid in puuid_list_6:
    # Get the match list for the current user
    response = requests.get(f'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=10',
                            headers={'X-Riot-Token': api_key})
    
    # Check if the response was successful
    if response.status_code != 200:
        print(f'Error getting match list for puuid {puuid}: {response.text}')
        continue
    
    # Extract up to 10 match IDs for the current user
    match_ids_6 = response.json()
    
    # Add the match IDs to the final list
    match_id_list_6.extend(match_ids_6)
    
    # Sleep for a short time to avoid hitting the rate limit
    time.sleep(2)

print(match_id_list_6)

In [ ]:
# csv 파일로 저장
import csv
f = open('matchid_6(0326).csv', 'w')
writer = csv.writer(f)
writer.writerow(match_id_list_6)
f.close()

### **match id 7 뽑고 저장**

In [ ]:
# 그냥 puuid_list 에 대해 적용
match_id_list_7 = []

# Define the maximum number of requests per second and the time to wait between each batch of requests
MAX_REQUESTS_PER_SECOND = 5
WAIT_TIME_BETWEEN_BATCHES = 1 / MAX_REQUESTS_PER_SECOND

# Loop through each puuid in the list
for puuid in puuid_list_7:
    # Get the match list for the current user
    response = requests.get(f'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=10',
                            headers={'X-Riot-Token': api_key})
    
    # Check if the response was successful
    if response.status_code != 200:
        print(f'Error getting match list for puuid {puuid}: {response.text}')
        continue
    
    # Extract up to 10 match IDs for the current user
    match_ids_7 = response.json()
    
    # Add the match IDs to the final list
    match_id_list_7.extend(match_ids_7)
    
    # Sleep for a short time to avoid hitting the rate limit
    time.sleep(2)

print(match_id_list_7)

In [ ]:
# csv 파일로 저장
import csv
f = open('matchid_7(0326).csv', 'w')
writer = csv.writer(f)
writer.writerow(match_id_list_7)
f.close()

### **match id 8 뽑고 저장**

In [ ]:
# 그냥 puuid_list 에 대해 적용
match_id_list_8 = []

# Define the maximum number of requests per second and the time to wait between each batch of requests
MAX_REQUESTS_PER_SECOND = 5
WAIT_TIME_BETWEEN_BATCHES = 1 / MAX_REQUESTS_PER_SECOND

# Loop through each puuid in the list
for puuid in puuid_list_8:
    # Get the match list for the current user
    response = requests.get(f'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=10',
                            headers={'X-Riot-Token': api_key})
    
    # Check if the response was successful
    if response.status_code != 200:
        print(f'Error getting match list for puuid {puuid}: {response.text}')
        continue
    
    # Extract up to 10 match IDs for the current user
    match_ids_8 = response.json()
    
    # Add the match IDs to the final list
    match_id_list_8.extend(match_ids_8)
    
    # Sleep for a short time to avoid hitting the rate limit
    time.sleep(2)

print(match_id_list_8)

In [ ]:
# csv 파일로 저장
import csv
f = open('matchid_8(0326).csv', 'w')
writer = csv.writer(f)
writer.writerow(match_id_list_8)
f.close()

### **match id 9 뽑고 저장** 

In [ ]:
# 그냥 puuid_list 에 대해 적용
match_id_list_9 = []

# Define the maximum number of requests per second and the time to wait between each batch of requests
MAX_REQUESTS_PER_SECOND = 5
WAIT_TIME_BETWEEN_BATCHES = 1 / MAX_REQUESTS_PER_SECOND

# Loop through each puuid in the list
for puuid in puuid_list_9:
    # Get the match list for the current user
    response = requests.get(f'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=10',
                            headers={'X-Riot-Token': api_key})
    
    # Check if the response was successful
    if response.status_code != 200:
        print(f'Error getting match list for puuid {puuid}: {response.text}')
        continue
    
    # Extract up to 10 match IDs for the current user
    match_ids_9 = response.json()
    
    # Add the match IDs to the final list
    match_id_list_9.extend(match_ids_9)
    
    # Sleep for a short time to avoid hitting the rate limit
    time.sleep(2)

print(match_id_list_9)

In [ ]:
# csv 파일로 저장
import csv
f = open('matchid_9(0326).csv', 'w')
writer = csv.writer(f)
writer.writerow(match_id_list_9)
f.close()

### **match id 10 뽑고 저장** 

In [ ]:
# 그냥 puuid_list 에 대해 적용
match_id_list_10 = []

# Define the maximum number of requests per second and the time to wait between each batch of requests
MAX_REQUESTS_PER_SECOND = 5
WAIT_TIME_BETWEEN_BATCHES = 1 / MAX_REQUESTS_PER_SECOND

# Loop through each puuid in the list
for puuid in puuid_list_10:
    # Get the match list for the current user
    response = requests.get(f'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=10',
                            headers={'X-Riot-Token': api_key})
    
    # Check if the response was successful
    if response.status_code != 200:
        print(f'Error getting match list for puuid {puuid}: {response.text}')
        continue
    
    # Extract up to 10 match IDs for the current user
    match_ids_10 = response.json()
    
    # Add the match IDs to the final list
    match_id_list_10.extend(match_ids_10)
    
    # Sleep for a short time to avoid hitting the rate limit
    time.sleep(2)

print(match_id_list_10)

In [ ]:
# csv 파일로 저장
import csv
f = open('matchid_10(0326).csv', 'w')
writer = csv.writer(f)
writer.writerow(match_id_list_10)
f.close()

**만들어진 모든 match id를 순서대로 합친다.** 

In [ ]:
match_id_list = match_id_list_1 + match_id_list_2 + match_id_list_3 + match_id_list_4 + match_id_list_5 + match_id_list_6 + match_id_list_7 + match_id_list_8 + match_id_list_9 + match_id_list_10  

In [ ]:
len(match_id_list)

20000

In [ ]:
for i in range(1, 11):
    list_name = f'match_id_list_{i}'
    sublist = globals()[list_name]
    print(len(sublist))

2000
2000
2000
2000
2000
2000
2000
2000
2000
2000


### **총 match id 리스트를 csv 형태로 저장한다.**

In [ ]:
# csv 파일로 저장
import csv
f = open('matchid(0326).csv', 'w')
writer = csv.writer(f)
writer.writerow(match_id_list)
f.close()

In [ ]:
# csv 파일을 리스트로 불러오기 
import csv 
match_id_list = []

with open('/content/drive/MyDrive/LOL_ANALYSIS_0323/matchid(0326).csv','r',newline='') as f:
    reader = csv.reader(f)
    for row in reader:
        match_id_list.append(row)

print(match_id_list)

In [ ]:
# 중복 괄호 제거용 코드
a = []  
a.extend(match_id_list[0])  
match_id_list = a
print(match_id_list)

### **Match id 리스트와 puuid를 하나로 합친 데이터프레임 만들기**
* 이 때, puuid 하나당 match id 10개 이므로, puuid_list 의 각 원소가 10번씩 반복되도록 하는 리스트를 새로 생성해준다. 

In [ ]:
from itertools import chain, repeat

puuid_list_10times = list(chain.from_iterable((repeat(number, 10) for number in puuid_list))) # 각 puuid 10번 반복한 리스트 
print(puuid_list_10times)   # check 
print(len(puuid_list_10times))

* puuid 와 match id 를 각 열로 하는 데이터프레임을 만든다

In [57]:
import pandas as pd
# create a DataFrame with puuid and match_id as column
match_id_df = pd.DataFrame()
match_id_df['puuid'] = puuid_list_10times
match_id_df['match_id'] = match_id_list
print(match_id_df.head(20))

                                                puuid       match_id
0   ev9cR3_z9HIDe-BbdMP8KcSQb1micGugw_ogBcCWUdUI4c...  KR_6417459659
1   ev9cR3_z9HIDe-BbdMP8KcSQb1micGugw_ogBcCWUdUI4c...  KR_6417367261
2   ev9cR3_z9HIDe-BbdMP8KcSQb1micGugw_ogBcCWUdUI4c...  KR_6417315320
3   ev9cR3_z9HIDe-BbdMP8KcSQb1micGugw_ogBcCWUdUI4c...  KR_6415496326
4   ev9cR3_z9HIDe-BbdMP8KcSQb1micGugw_ogBcCWUdUI4c...  KR_6415488148
5   ev9cR3_z9HIDe-BbdMP8KcSQb1micGugw_ogBcCWUdUI4c...  KR_6414254440
6   ev9cR3_z9HIDe-BbdMP8KcSQb1micGugw_ogBcCWUdUI4c...  KR_6414200709
7   ev9cR3_z9HIDe-BbdMP8KcSQb1micGugw_ogBcCWUdUI4c...  KR_6413832314
8   ev9cR3_z9HIDe-BbdMP8KcSQb1micGugw_ogBcCWUdUI4c...  KR_6413781744
9   ev9cR3_z9HIDe-BbdMP8KcSQb1micGugw_ogBcCWUdUI4c...  KR_6413732650
10  7dGrIHqteZzlicQ9pgm80swBRNzmV16qYFeVnmRZBYrkot...  KR_6412274821
11  7dGrIHqteZzlicQ9pgm80swBRNzmV16qYFeVnmRZBYrkot...  KR_6412135329
12  7dGrIHqteZzlicQ9pgm80swBRNzmV16qYFeVnmRZBYrkot...  KR_6410507032
13  7dGrIHqteZzlicQ9pgm80swBRNzmV1

* puuid - match id 데이터프레임 csv 로 저장하기 

In [9]:
match_id_df.to_csv('puuidXmatchid_df(0326).csv', header=True, index=False)

In [9]:
# 데이터프레임 불러오기 
match_id_df = pd.read_csv('/content/drive/MyDrive/LOL_ANALYSIS_0323/puuidXmatchid_df(0326).csv')
match_id_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   puuid     20000 non-null  object
 1   match_id  20000 non-null  object
dtypes: object(2)
memory usage: 312.6+ KB


# **Match Data 뽑기**

### 데이터 양이 엄청 많다. 한번에 다 하려면 시간이 너무 오래걸려서 중간에 끊길 위험이 있으므로 여러번 나눠서 저장한다. 

* 데이터프레임을 다음과 같이 10덩어리로 나눈다. 

In [10]:
match_id_df_1 = match_id_df.iloc[0:2000,:]
match_id_df_2 = match_id_df.iloc[2000:4000,:]
match_id_df_3 = match_id_df.iloc[4000:6000,:]
match_id_df_4 = match_id_df.iloc[6000:8000,:]
match_id_df_5 = match_id_df.iloc[8000:10000,:]
match_id_df_6 = match_id_df.iloc[10000:12000,:]
match_id_df_7 = match_id_df.iloc[12000:14000,:]
match_id_df_8 = match_id_df.iloc[14000:16000,:]
match_id_df_9 = match_id_df.iloc[16000:18000,:]
match_id_df_10 = match_id_df.iloc[18000:20000,:]
#작동 잘되는지 테스트용 데이터셋
match_id_df_mini = match_id_df.iloc[0:10,:]

### 위에서 만들어둔 미니 데이터셋 match_id_df_mini 를 사용해서 테스트해본다(잘 잘동하는지)

In [90]:
for i in range(0,10): #########길이 수정 요망########
    pair = match_id_df_mini.iloc[i,:].to_list()
    puuid = pair[0]
    match_id = pair[1]
    url = f'https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}'

    # Make the Riot API call
    response = requests.get(url)
    response_json = response.json()

    assists = None  # set a default value for assists
    for participant in response_json['info']['participants']:
      if participant['puuid'] == puuid and participant['teamPosition'] == 'MIDDLE':
        print(participant['puuid'])
        print(participant['kills'])

ev9cR3_z9HIDe-BbdMP8KcSQb1micGugw_ogBcCWUdUI4cyrrV6Np3ozKtFexJIkJ0gc47CCsl6AUg
11
ev9cR3_z9HIDe-BbdMP8KcSQb1micGugw_ogBcCWUdUI4cyrrV6Np3ozKtFexJIkJ0gc47CCsl6AUg
9
ev9cR3_z9HIDe-BbdMP8KcSQb1micGugw_ogBcCWUdUI4cyrrV6Np3ozKtFexJIkJ0gc47CCsl6AUg
2
ev9cR3_z9HIDe-BbdMP8KcSQb1micGugw_ogBcCWUdUI4cyrrV6Np3ozKtFexJIkJ0gc47CCsl6AUg
8
ev9cR3_z9HIDe-BbdMP8KcSQb1micGugw_ogBcCWUdUI4cyrrV6Np3ozKtFexJIkJ0gc47CCsl6AUg
6


In [91]:
region = 'asia'
# Empty lists to store the collected data
# game_mode_list = [] 얘넨 나중에
# game_type_list = []
assists_list = []
kills_list = []
deaths_list = []
champExp_list = [] #champ_experience_list
champName_list = [] #champion_name_list
indiPos_list = [] #individual_position_list
lane_list = []
role_list = []
gold_list = []   #gold_earned_list
teamPos_list = []
tDDealt_list = []  #total_damage_dealt_list
tDDealtChamp_list = [] #total_damage_dealt_to_champions_list
tDTaken_list = []  #total_damage_taken_list
tMKilled_list = []  #total_minions_killed_list
turKills_list = []  #turret_kills_list
win_list = []


for i in range(0,10): #########길이 수정 요망########
    pair = match_id_df_mini.iloc[i,:].to_list()
    puuid = pair[0]
    match_id = pair[1]
    url = f'https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}'

    # Make the Riot API call
    response = requests.get(url)
    response_json = response.json()

    
    kills = None 
    assists = None  # set a default value
    deaths = None 
    champExp = None 
    champName = None 
    indiPos = None 
    teamPos = None 
    lane = None 
    role = None 
    gold = None 
    tDDealt = None 
    tDDealtChamp = None 
    tDTaken = None 
    tMKilled = None
    turKills = None
    win = None
    for participant in response_json['info']['participants']:
      if participant['puuid'] == puuid and participant['teamPosition'] == 'MIDDLE': #게임에 참여한 참가자들 중 위에서 우리가 구한 puuid와 puuid 가 일치하고 individualPosition이 MIDDLE인 participants에 대해서만 정보 추출 
        kills = participant['kills']
        assists = participant['assists']
        deaths = participant["deaths"]
        champExp = participant["champExperience"]
        champName = participant["championName"]
        indiPos = participant["individualPosition"]
        teamPos = participant["teamPosition"]
        lane = participant["lane"]
        role = participant["role"]
        gold = participant["goldEarned"]
        tDDealt = participant["totalDamageDealt"]
        tDDealtChamp = participant["totalDamageDealtToChampions"]
        tDTaken = participant["totalDamageTaken"]
        tMKilled = participant["totalMinionsKilled"]
        turKills = participant["turretKills"]
        win = participant["win"]
        break
    assists_list.append(assists)  # append assists only if it was set to a non-default value : 근데 이렇게 하면 결측치는 그냥 삭제되어버림. 고쳐야 할듯?
    kills_list.append(kills)
    deaths_list.append(deaths)
    champExp_list.append(champExp)
    champName_list.append(champName)
    indiPos_list.append(indiPos)
    teamPos_list.append(teamPos)
    lane_list.append(lane)
    role_list.append(role)
    gold_list.append(gold)
    tDDealt_list.append(tDDealt)
    tDDealtChamp_list.append(tDDealtChamp)
    tDTaken_list.append(tDTaken)
    tMKilled_list.append(tMKilled)
    turKills_list.append(turKills)
    win_list.append(win)

    time.sleep(1)

In [92]:
# 잘 추출되었는지 확인
print(len(assists_list))
print(len(kills_list))
print(len(deaths_list))
print(len(champExp_list))
print(len(champName_list))
print(len(indiPos_list))
print(len(teamPos_list))
print(len(lane_list))
print(len(role_list))
print(len(gold_list))
print(len(tDDealt_list))
print(len(tDDealtChamp_list))
print(len(tDTaken_list))
print(len(tMKilled_list))
print(len(turKills_list))
print(len(win_list))

10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10


In [93]:
# 잘 추출되었는지 확인
print(assists_list)
print(kills_list)
print(deaths_list)
print(champExp_list)
print(champName_list)
print(indiPos_list)
print(teamPos_list)
print(lane_list)
print(role_list)
print(gold_list)
print(tDDealt_list)
print(tDDealtChamp_list)
print(tDTaken_list)
print(tMKilled_list)
print(turKills_list)
print(win_list)

[None, None, None, None, None, 10, 2, 0, 5, 2]
[None, None, None, None, None, 11, 9, 2, 8, 6]
[None, None, None, None, None, 2, 0, 3, 2, 3]
[None, None, None, None, None, 13110, 8716, 7995, 12688, 21625]
[None, None, None, None, None, 'Sett', 'Sett', 'Garen', 'Garen', 'Sett']
[None, None, None, None, None, 'MIDDLE', 'MIDDLE', 'MIDDLE', 'MIDDLE', 'MIDDLE']
[None, None, None, None, None, 'MIDDLE', 'MIDDLE', 'MIDDLE', 'MIDDLE', 'MIDDLE']
[None, None, None, None, None, 'MIDDLE', 'NONE', 'NONE', 'MIDDLE', 'MIDDLE']
[None, None, None, None, None, 'SOLO', 'SUPPORT', 'SUPPORT', 'SOLO', 'DUO']
[None, None, None, None, None, 13096, 10750, 5659, 11262, 15372]
[None, None, None, None, None, 129264, 99882, 58604, 99840, 240132]
[None, None, None, None, None, 28486, 10026, 5056, 18053, 15784]
[None, None, None, None, None, 30670, 11422, 9121, 20400, 40154]
[None, None, None, None, None, 152, 132, 114, 147, 278]
[None, None, None, None, None, 2, 3, 0, 3, 3]
[None, None, None, None, None, True, True, 

In [94]:
# 리스트를 column으로 해서 데이터프레임 만들기
mini_df = pd.DataFrame()
mini_df['assists'] = assists_list
mini_df['kills'] = kills_list
mini_df['deaths'] = deaths_list
mini_df['champExp'] = champExp_list #champ_experience_list
mini_df['champName'] = champName_list #champion_name_list
mini_df['indiPos'] = indiPos_list #individual_position_list
mini_df['lane'] = lane_list
mini_df['role'] = role_list 
mini_df['gold'] = gold_list    #gold_earned_list
mini_df['teamPos'] = teamPos_list 
mini_df['tDDealt'] = tDDealt_list  #total_damage_dealt_list
mini_df['tDDealtChamp'] = tDDealtChamp_list #total_damage_dealt_to_champions_list
mini_df['tDTaken'] = tDTaken_list  #total_damage_taken_list
mini_df['tMKilled'] = tMKilled_list   #total_minions_killed_list
mini_df['turKills'] = turKills_list   #turret_kills_list
mini_df['win'] = win_list 

In [95]:
mini_df

,assists,kills,deaths,champExp,champName,indiPos,lane,role,gold,teamPos,tDDealt,tDDealtChamp,tDTaken,tMKilled,turKills,win
0,NaN,NaN,NaN,NaN,None,None,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,None
1,NaN,NaN,NaN,NaN,None,None,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,None
2,NaN,NaN,NaN,NaN,None,None,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,None
3,NaN,NaN,NaN,NaN,None,None,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,None
4,NaN,NaN,NaN,NaN,None,None,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,None
5,10.0,11.0,2.0,13110.0,Sett,MIDDLE,MIDDLE,SOLO,13096.0,MIDDLE,129264.0,28486.0,30670.0,152.0,2.0,True
6,2.0,9.0,0.0,8716.0,Sett,MIDDLE,NONE,SUPPORT,10750.0,MIDDLE,99882.0,10026.0,11422.0,132.0,3.0,True
7,0.0,2.0,3.0,7995.0,Garen,MIDDLE,NONE,SUPPORT,5659.0,MIDDLE,58604.0,5056.0,9121.0,114.0,0.0,False
8,5.0,8.0,2.0,12688.0,Garen,MIDDLE,MIDDLE,SOLO,11262.0,MIDDLE,99840.0,18053.0,20400.0,147.0,3.0,True
9,2.0,6.0,3.0,21625.0,Sett,MIDDLE,MIDDLE,DUO,15372.0,MIDDLE,240132.0,15784.0,40154.0,278.0,3.0,True


# **테스트 결과 잘 나옴 >> match_id_df_1 부터 match_id_df_10까지 노가다 시작**

### **MATCH 1**

In [96]:
region = 'asia'
# Empty lists to store the collected data
# game_mode_list = [] 얘넨 나중에
# game_type_list = []
assists_list = []
kills_list = []
deaths_list = []
champExp_list = [] #champ_experience_list
champName_list = [] #champion_name_list
indiPos_list = [] #individual_position_list
lane_list = []
role_list = []
gold_list = []   #gold_earned_list
teamPos_list = []
tDDealt_list = []  #total_damage_dealt_list
tDDealtChamp_list = [] #total_damage_dealt_to_champions_list
tDTaken_list = []  #total_damage_taken_list
tMKilled_list = []  #total_minions_killed_list
turKills_list = []  #turret_kills_list
win_list = []


for i in range(0,2000): #########길이 수정 요망########
    pair = match_id_df_1.iloc[i,:].to_list()
    puuid = pair[0]
    match_id = pair[1]
    url = f'https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}'

    # Make the Riot API call
    response = requests.get(url)
    response_json = response.json()

    
    kills = None 
    assists = None  # set a default value
    deaths = None 
    champExp = None 
    champName = None 
    indiPos = None 
    teamPos = None 
    lane = None 
    role = None 
    gold = None 
    tDDealt = None 
    tDDealtChamp = None 
    tDTaken = None 
    tMKilled = None
    turKills = None
    win = None
    for participant in response_json['info']['participants']:
      if participant['puuid'] == puuid and participant['teamPosition'] == 'MIDDLE': #게임에 참여한 참가자들 중 위에서 우리가 구한 puuid와 puuid 가 일치하고 individualPosition이 MIDDLE인 participants에 대해서만 정보 추출 
        kills = participant['kills']
        assists = participant['assists']
        deaths = participant["deaths"]
        champExp = participant["champExperience"]
        champName = participant["championName"]
        indiPos = participant["individualPosition"]
        teamPos = participant["teamPosition"]
        lane = participant["lane"]
        role = participant["role"]
        gold = participant["goldEarned"]
        tDDealt = participant["totalDamageDealt"]
        tDDealtChamp = participant["totalDamageDealtToChampions"]
        tDTaken = participant["totalDamageTaken"]
        tMKilled = participant["totalMinionsKilled"]
        turKills = participant["turretKills"]
        win = participant["win"]
        break
    assists_list.append(assists)  # append assists only if it was set to a non-default value : 근데 이렇게 하면 결측치는 그냥 삭제되어버림. 고쳐야 할듯?
    kills_list.append(kills)
    deaths_list.append(deaths)
    champExp_list.append(champExp)
    champName_list.append(champName)
    indiPos_list.append(indiPos)
    teamPos_list.append(teamPos)
    lane_list.append(lane)
    role_list.append(role)
    gold_list.append(gold)
    tDDealt_list.append(tDDealt)
    tDDealtChamp_list.append(tDDealtChamp)
    tDTaken_list.append(tDTaken)
    tMKilled_list.append(tMKilled)
    turKills_list.append(turKills)
    win_list.append(win)

    time.sleep(1)

In [ ]:
#길이
print(len(assists_list))
print(len(kills_list))
print(len(deaths_list))
print(len(champExp_list))
print(len(champName_list))
print(len(indiPos_list))
print(len(teamPos_list))
print(len(lane_list))
print(len(role_list))
print(len(gold_list))
print(len(tDDealt_list))
print(len(tDDealtChamp_list))
print(len(tDTaken_list))
print(len(tMKilled_list))
print(len(turKills_list))
print(len(win_list))

#내용
print(assists_list)
print(kills_list)
print(deaths_list)
print(champExp_list)
print(champName_list)
print(indiPos_list)
print(teamPos_list)
print(lane_list)
print(role_list)
print(gold_list)
print(tDDealt_list)
print(tDDealtChamp_list)
print(tDTaken_list)
print(tMKilled_list)
print(turKills_list)
print(win_list)

In [100]:
matchdata_1 = pd.DataFrame()
matchdata_1['assists'] = assists_list
matchdata_1['kills'] = kills_list
matchdata_1['deaths'] = deaths_list
matchdata_1['champExp'] = champExp_list #champ_experience_list
matchdata_1['champName'] = champName_list #champion_name_list
matchdata_1['indiPos'] = indiPos_list #individual_position_list
matchdata_1['lane'] = lane_list
matchdata_1['role'] = role_list 
matchdata_1['gold'] = gold_list    #gold_earned_list
matchdata_1['teamPos'] = teamPos_list 
matchdata_1['tDDealt'] = tDDealt_list  #total_damage_dealt_list
matchdata_1['tDDealtChamp'] = tDDealtChamp_list #total_damage_dealt_to_champions_list
matchdata_1['tDTaken'] = tDTaken_list  #total_damage_taken_list
matchdata_1['tMKilled'] = tMKilled_list   #total_minions_killed_list
matchdata_1['turKills'] = turKills_list   #turret_kills_list
matchdata_1['win'] = win_list 

In [ ]:
matchdata_1.info()

In [ ]:
matchdata_1.head(10)

In [106]:
matchdata_1.to_csv('matchdata_1(0326).csv', header=True, index=False)

### **MATCH 2**

In [107]:
region = 'asia'
# Empty lists to store the collected data
# game_mode_list = [] 얘넨 나중에
# game_type_list = []

assists_list = []
kills_list = []
deaths_list = []
champExp_list = [] #champ_experience_list
champName_list = [] #champion_name_list
indiPos_list = [] #individual_position_list
lane_list = []
role_list = []
gold_list = []   #gold_earned_list
teamPos_list = []
tDDealt_list = []  #total_damage_dealt_list
tDDealtChamp_list = [] #total_damage_dealt_to_champions_list
tDTaken_list = []  #total_damage_taken_list
tMKilled_list = []  #total_minions_killed_list
turKills_list = []  #turret_kills_list
win_list = []


for i in range(0,2000): #########길이 수정 요망########
    pair = match_id_df_2.iloc[i,:].to_list()
    puuid = pair[0]
    match_id = pair[1]
    url = f'https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}'

    # Make the Riot API call
    response = requests.get(url)
    response_json = response.json()

    
    kills = None 
    assists = None  # set a default value
    deaths = None 
    champExp = None 
    champName = None 
    indiPos = None 
    teamPos = None 
    lane = None 
    role = None 
    gold = None 
    tDDealt = None 
    tDDealtChamp = None 
    tDTaken = None 
    tMKilled = None
    turKills = None
    win = None
    for participant in response_json['info']['participants']:
      if participant['puuid'] == puuid and participant['teamPosition'] == 'MIDDLE': #게임에 참여한 참가자들 중 위에서 우리가 구한 puuid와 puuid 가 일치하고 individualPosition이 MIDDLE인 participants에 대해서만 정보 추출 
        kills = participant['kills']
        assists = participant['assists']
        deaths = participant["deaths"]
        champExp = participant["champExperience"]
        champName = participant["championName"]
        indiPos = participant["individualPosition"]
        teamPos = participant["teamPosition"]
        lane = participant["lane"]
        role = participant["role"]
        gold = participant["goldEarned"]
        tDDealt = participant["totalDamageDealt"]
        tDDealtChamp = participant["totalDamageDealtToChampions"]
        tDTaken = participant["totalDamageTaken"]
        tMKilled = participant["totalMinionsKilled"]
        turKills = participant["turretKills"]
        win = participant["win"]
        break
    assists_list.append(assists)  # append assists only if it was set to a non-default value : 근데 이렇게 하면 결측치는 그냥 삭제되어버림. 고쳐야 할듯?
    kills_list.append(kills)
    deaths_list.append(deaths)
    champExp_list.append(champExp)
    champName_list.append(champName)
    indiPos_list.append(indiPos)
    teamPos_list.append(teamPos)
    lane_list.append(lane)
    role_list.append(role)
    gold_list.append(gold)
    tDDealt_list.append(tDDealt)
    tDDealtChamp_list.append(tDDealtChamp)
    tDTaken_list.append(tDTaken)
    tMKilled_list.append(tMKilled)
    turKills_list.append(turKills)
    win_list.append(win)

    time.sleep(1)

In [ ]:
#길이
print(len(assists_list))
print(len(kills_list))
print(len(deaths_list))
print(len(champExp_list))
print(len(champName_list))
print(len(indiPos_list))
print(len(teamPos_list))
print(len(lane_list))
print(len(role_list))
print(len(gold_list))
print(len(tDDealt_list))
print(len(tDDealtChamp_list))
print(len(tDTaken_list))
print(len(tMKilled_list))
print(len(turKills_list))
print(len(win_list))

#내용
print(assists_list)
print(kills_list)
print(deaths_list)
print(champExp_list)
print(champName_list)
print(indiPos_list)
print(teamPos_list)
print(lane_list)
print(role_list)
print(gold_list)
print(tDDealt_list)
print(tDDealtChamp_list)
print(tDTaken_list)
print(tMKilled_list)
print(turKills_list)
print(win_list)

In [112]:
matchdata_2 = pd.DataFrame()
matchdata_2['assists'] = assists_list
matchdata_2['kills'] = kills_list
matchdata_2['deaths'] = deaths_list
matchdata_2['champExp'] = champExp_list #champ_experience_list
matchdata_2['champName'] = champName_list #champion_name_list
matchdata_2['indiPos'] = indiPos_list #individual_position_list
matchdata_2['lane'] = lane_list
matchdata_2['role'] = role_list 
matchdata_2['gold'] = gold_list    #gold_earned_list
matchdata_2['teamPos'] = teamPos_list 
matchdata_2['tDDealt'] = tDDealt_list  #total_damage_dealt_list
matchdata_2['tDDealtChamp'] = tDDealtChamp_list #total_damage_dealt_to_champions_list
matchdata_2['tDTaken'] = tDTaken_list  #total_damage_taken_list
matchdata_2['tMKilled'] = tMKilled_list   #total_minions_killed_list
matchdata_2['turKills'] = turKills_list   #turret_kills_list
matchdata_2['win'] = win_list 

In [ ]:
matchdata_2.info()

In [ ]:
matchdata_2.head(30)

In [117]:
matchdata_2.to_csv('matchdata_2(0326).csv', header=True, index=False)

### **MATCH 3**

In [8]:



region = 'asia'
# Empty lists to store the collected data
# game_mode_list = [] 얘넨 나중에
# game_type_list = []

assists_list = []
kills_list = []
deaths_list = []
champExp_list = [] #champ_experience_list
champName_list = [] #champion_name_list
indiPos_list = [] #individual_position_list
lane_list = []
role_list = []
gold_list = []   #gold_earned_list
teamPos_list = []
tDDealt_list = []  #total_damage_dealt_list
tDDealtChamp_list = [] #total_damage_dealt_to_champions_list
tDTaken_list = []  #total_damage_taken_list
tMKilled_list = []  #total_minions_killed_list
turKills_list = []  #turret_kills_list
win_list = []


for i in range(0,2000):
    pair = match_id_df_3.iloc[i,:].to_list()#-'-'-'-'-'-'-'--'-'-'-'
    puuid = pair[0]
    match_id = pair[1]
    url = f'https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}'

    # Make the Riot API call
    response = requests.get(url)
    response_json = response.json()

    
    kills = None 
    assists = None  # set a default value
    deaths = None 
    champExp = None 
    champName = None 
    indiPos = None 
    teamPos = None 
    lane = None 
    role = None 
    gold = None 
    tDDealt = None 
    tDDealtChamp = None 
    tDTaken = None 
    tMKilled = None
    turKills = None
    win = None
    for participant in response_json['info']['participants']:
      if participant['puuid'] == puuid and participant['teamPosition'] == 'MIDDLE': #게임에 참여한 참가자들 중 위에서 우리가 구한 puuid와 puuid 가 일치하고 individualPosition이 MIDDLE인 participants에 대해서만 정보 추출 
        kills = participant['kills']
        assists = participant['assists']
        deaths = participant["deaths"]
        champExp = participant["champExperience"]
        champName = participant["championName"]
        indiPos = participant["individualPosition"]
        teamPos = participant["teamPosition"]
        lane = participant["lane"]
        role = participant["role"]
        gold = participant["goldEarned"]
        tDDealt = participant["totalDamageDealt"]
        tDDealtChamp = participant["totalDamageDealtToChampions"]
        tDTaken = participant["totalDamageTaken"]
        tMKilled = participant["totalMinionsKilled"]
        turKills = participant["turretKills"]
        win = participant["win"]
        break
    assists_list.append(assists)  # append assists only if it was set to a non-default value : 근데 이렇게 하면 결측치는 그냥 삭제되어버림. 고쳐야 할듯?
    kills_list.append(kills)
    deaths_list.append(deaths)
    champExp_list.append(champExp)
    champName_list.append(champName)
    indiPos_list.append(indiPos)
    teamPos_list.append(teamPos)
    lane_list.append(lane)
    role_list.append(role)
    gold_list.append(gold)
    tDDealt_list.append(tDDealt)
    tDDealtChamp_list.append(tDDealtChamp)
    tDTaken_list.append(tDTaken)
    tMKilled_list.append(tMKilled)
    turKills_list.append(turKills)
    win_list.append(win)

    time.sleep(1)

In [ ]:
#길이
print(len(assists_list))
print(len(kills_list))
print(len(deaths_list))
print(len(champExp_list))
print(len(champName_list))
print(len(indiPos_list))
print(len(teamPos_list))
print(len(lane_list))
print(len(role_list))
print(len(gold_list))
print(len(tDDealt_list))
print(len(tDDealtChamp_list))
print(len(tDTaken_list))
print(len(tMKilled_list))
print(len(turKills_list))
print(len(win_list))

#내용
print(assists_list)
print(kills_list)
print(deaths_list)
print(champExp_list)
print(champName_list)
print(indiPos_list)
print(teamPos_list)
print(lane_list)
print(role_list)
print(gold_list)
print(tDDealt_list)
print(tDDealtChamp_list)
print(tDTaken_list)
print(tMKilled_list)
print(turKills_list)
print(win_list)


In [10]:
matchdata_3 = pd.DataFrame()
matchdata_3['assists'] = assists_list
matchdata_3['kills'] = kills_list
matchdata_3['deaths'] = deaths_list
matchdata_3['champExp'] = champExp_list #champ_experience_list
matchdata_3['champName'] = champName_list #champion_name_list
matchdata_3['indiPos'] = indiPos_list #individual_position_list
matchdata_3['lane'] = lane_list
matchdata_3['role'] = role_list 
matchdata_3['gold'] = gold_list    #gold_earned_list
matchdata_3['teamPos'] = teamPos_list 
matchdata_3['tDDealt'] = tDDealt_list  #total_damage_dealt_list
matchdata_3['tDDealtChamp'] = tDDealtChamp_list #total_damage_dealt_to_champions_list
matchdata_3['tDTaken'] = tDTaken_list  #total_damage_taken_list
matchdata_3['tMKilled'] = tMKilled_list   #total_minions_killed_list
matchdata_3['turKills'] = turKills_list   #turret_kills_list
matchdata_3['win'] = win_list 

In [ ]:
print(matchdata_3.info())

print(matchdata_3.head(20))

In [12]:
matchdata_3.to_csv('matchdata_3(0326).csv', header=True, index=False)


### **MATCH 4**

In [13]:
region = 'asia'
# Empty lists to store the collected data
# game_mode_list = [] 얘넨 나중에
# game_type_list = []

assists_list = []
kills_list = []
deaths_list = []
champExp_list = [] #champ_experience_list
champName_list = [] #champion_name_list
indiPos_list = [] #individual_position_list
lane_list = []
role_list = []
gold_list = []   #gold_earned_list
teamPos_list = []
tDDealt_list = []  #total_damage_dealt_list
tDDealtChamp_list = [] #total_damage_dealt_to_champions_list
tDTaken_list = []  #total_damage_taken_list
tMKilled_list = []  #total_minions_killed_list
turKills_list = []  #turret_kills_list
win_list = []


for i in range(0,2000): #########길이 수정 요망########
    pair = match_id_df_4.iloc[i,:].to_list()
    puuid = pair[0]
    match_id = pair[1]
    url = f'https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}'

    # Make the Riot API call
    response = requests.get(url)
    response_json = response.json()

    
    kills = None 
    assists = None  # set a default value
    deaths = None 
    champExp = None 
    champName = None 
    indiPos = None 
    teamPos = None 
    lane = None 
    role = None 
    gold = None 
    tDDealt = None 
    tDDealtChamp = None 
    tDTaken = None 
    tMKilled = None
    turKills = None
    win = None
    for participant in response_json['info']['participants']:
      if participant['puuid'] == puuid and participant['teamPosition'] == 'MIDDLE': #게임에 참여한 참가자들 중 위에서 우리가 구한 puuid와 puuid 가 일치하고 individualPosition이 MIDDLE인 participants에 대해서만 정보 추출 
        kills = participant['kills']
        assists = participant['assists']
        deaths = participant["deaths"]
        champExp = participant["champExperience"]
        champName = participant["championName"]
        indiPos = participant["individualPosition"]
        teamPos = participant["teamPosition"]
        lane = participant["lane"]
        role = participant["role"]
        gold = participant["goldEarned"]
        tDDealt = participant["totalDamageDealt"]
        tDDealtChamp = participant["totalDamageDealtToChampions"]
        tDTaken = participant["totalDamageTaken"]
        tMKilled = participant["totalMinionsKilled"]
        turKills = participant["turretKills"]
        win = participant["win"]
        break
    assists_list.append(assists)  # append assists only if it was set to a non-default value : 근데 이렇게 하면 결측치는 그냥 삭제되어버림. 고쳐야 할듯?
    kills_list.append(kills)
    deaths_list.append(deaths)
    champExp_list.append(champExp)
    champName_list.append(champName)
    indiPos_list.append(indiPos)
    teamPos_list.append(teamPos)
    lane_list.append(lane)
    role_list.append(role)
    gold_list.append(gold)
    tDDealt_list.append(tDDealt)
    tDDealtChamp_list.append(tDDealtChamp)
    tDTaken_list.append(tDTaken)
    tMKilled_list.append(tMKilled)
    turKills_list.append(turKills)
    win_list.append(win)

    time.sleep(1)


In [ ]:
#길이
print(len(assists_list))
print(len(kills_list))
print(len(deaths_list))
print(len(champExp_list))
print(len(champName_list))
print(len(indiPos_list))
print(len(teamPos_list))
print(len(lane_list))
print(len(role_list))
print(len(gold_list))
print(len(tDDealt_list))
print(len(tDDealtChamp_list))
print(len(tDTaken_list))
print(len(tMKilled_list))
print(len(turKills_list))
print(len(win_list))

#내용
print(assists_list)
print(kills_list)
print(deaths_list)
print(champExp_list)
print(champName_list)
print(indiPos_list)
print(teamPos_list)
print(lane_list)
print(role_list)
print(gold_list)
print(tDDealt_list)
print(tDDealtChamp_list)
print(tDTaken_list)
print(tMKilled_list)
print(turKills_list)
print(win_list)

In [ ]:
matchdata_4 = pd.DataFrame()
matchdata_4['assists'] = assists_list
matchdata_4['kills'] = kills_list
matchdata_4['deaths'] = deaths_list
matchdata_4['champExp'] = champExp_list #champ_experience_list
matchdata_4['champName'] = champName_list #champion_name_list
matchdata_4['indiPos'] = indiPos_list #individual_position_list
matchdata_4['lane'] = lane_list
matchdata_4['role'] = role_list 
matchdata_4['gold'] = gold_list    #gold_earned_list
matchdata_4['teamPos'] = teamPos_list 
matchdata_4['tDDealt'] = tDDealt_list  #total_damage_dealt_list
matchdata_4['tDDealtChamp'] = tDDealtChamp_list #total_damage_dealt_to_champions_list
matchdata_4['tDTaken'] = tDTaken_list  #total_damage_taken_list
matchdata_4['tMKilled'] = tMKilled_list   #total_minions_killed_list
matchdata_4['turKills'] = turKills_list   #turret_kills_list
matchdata_4['win'] = win_list 

print(matchdata_4.info())

print(matchdata_4.head(20))


In [16]:
matchdata_4.to_csv('matchdata_4(0326).csv', header=True, index=False)

### **MATCH 5**

In [28]:
region = 'asia'
# Empty lists to store the collected data
# game_mode_list = [] 얘넨 나중에
# game_type_list = []

assists_list = []
kills_list = []
deaths_list = []
champExp_list = [] #champ_experience_list
champName_list = [] #champion_name_list
indiPos_list = [] #individual_position_list
lane_list = []
role_list = []
gold_list = []   #gold_earned_list
teamPos_list = []
tDDealt_list = []  #total_damage_dealt_list
tDDealtChamp_list = [] #total_damage_dealt_to_champions_list
tDTaken_list = []  #total_damage_taken_list
tMKilled_list = []  #total_minions_killed_list
turKills_list = []  #turret_kills_list
win_list = []


for i in range(0,2000): 
    pair = match_id_df_5.iloc[i,:].to_list()#='='='='='='='='='='='='='='='==='=='='='='
    puuid = pair[0]
    match_id = pair[1]
    url = f'https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}'

    # Make the Riot API call
    response = requests.get(url)
    response_json = response.json()

    
    kills = None 
    assists = None  # set a default value
    deaths = None 
    champExp = None 
    champName = None 
    indiPos = None 
    teamPos = None 
    lane = None 
    role = None 
    gold = None 
    tDDealt = None 
    tDDealtChamp = None 
    tDTaken = None 
    tMKilled = None
    turKills = None
    win = None
    for participant in response_json['info']['participants']:
      if participant['puuid'] == puuid and participant['teamPosition'] == 'MIDDLE': #게임에 참여한 참가자들 중 위에서 우리가 구한 puuid와 puuid 가 일치하고 individualPosition이 MIDDLE인 participants에 대해서만 정보 추출 
        kills = participant['kills']
        assists = participant['assists']
        deaths = participant["deaths"]
        champExp = participant["champExperience"]
        champName = participant["championName"]
        indiPos = participant["individualPosition"]
        teamPos = participant["teamPosition"]
        lane = participant["lane"]
        role = participant["role"]
        gold = participant["goldEarned"]
        tDDealt = participant["totalDamageDealt"]
        tDDealtChamp = participant["totalDamageDealtToChampions"]
        tDTaken = participant["totalDamageTaken"]
        tMKilled = participant["totalMinionsKilled"]
        turKills = participant["turretKills"]
        win = participant["win"]
        break
    assists_list.append(assists)  # append assists only if it was set to a non-default value : 근데 이렇게 하면 결측치는 그냥 삭제되어버림. 고쳐야 할듯?
    kills_list.append(kills)
    deaths_list.append(deaths)
    champExp_list.append(champExp)
    champName_list.append(champName)
    indiPos_list.append(indiPos)
    teamPos_list.append(teamPos)
    lane_list.append(lane)
    role_list.append(role)
    gold_list.append(gold)
    tDDealt_list.append(tDDealt)
    tDDealtChamp_list.append(tDDealtChamp)
    tDTaken_list.append(tDTaken)
    tMKilled_list.append(tMKilled)
    turKills_list.append(turKills)
    win_list.append(win)

    time.sleep(1)


In [ ]:
#길이
print(len(assists_list))
print(len(kills_list))
print(len(deaths_list))
print(len(champExp_list))
print(len(champName_list))
print(len(indiPos_list))
print(len(teamPos_list))
print(len(lane_list))
print(len(role_list))
print(len(gold_list))
print(len(tDDealt_list))
print(len(tDDealtChamp_list))
print(len(tDTaken_list))
print(len(tMKilled_list))
print(len(turKills_list))
print(len(win_list))

#내용
print(assists_list)
print(kills_list)
print(deaths_list)
print(champExp_list)
print(champName_list)
print(indiPos_list)
print(teamPos_list)
print(lane_list)
print(role_list)
print(gold_list)
print(tDDealt_list)
print(tDDealtChamp_list)
print(tDTaken_list)
print(tMKilled_list)
print(turKills_list)
print(win_list)

In [ ]:
matchdata_5 = pd.DataFrame()
matchdata_5['assists'] = assists_list
matchdata_5['kills'] = kills_list
matchdata_5['deaths'] = deaths_list
matchdata_5['champExp'] = champExp_list #champ_experience_list
matchdata_5['champName'] = champName_list #champion_name_list
matchdata_5['indiPos'] = indiPos_list #individual_position_list
matchdata_5['lane'] = lane_list
matchdata_5['role'] = role_list 
matchdata_5['gold'] = gold_list    #gold_earned_list
matchdata_5['teamPos'] = teamPos_list 
matchdata_5['tDDealt'] = tDDealt_list  #total_damage_dealt_list
matchdata_5['tDDealtChamp'] = tDDealtChamp_list #total_damage_dealt_to_champions_list
matchdata_5['tDTaken'] = tDTaken_list  #total_damage_taken_list
matchdata_5['tMKilled'] = tMKilled_list   #total_minions_killed_list
matchdata_5['turKills'] = turKills_list   #turret_kills_list
matchdata_5['win'] = win_list 

print(matchdata_5.info())

print(matchdata_5.head(20))

In [33]:
matchdata_5.to_csv('matchdata_5(0326).csv', header=True, index=False)

### **MATCH 6**

In [47]:
region = 'asia'
# Empty lists to store the collected data
# game_mode_list = [] 얘넨 나중에
# game_type_list = []

assists_list = []
kills_list = []
deaths_list = []
champExp_list = [] #champ_experience_list
champName_list = [] #champion_name_list
indiPos_list = [] #individual_position_list
lane_list = []
role_list = []
gold_list = []   #gold_earned_list
teamPos_list = []
tDDealt_list = []  #total_damage_dealt_list
tDDealtChamp_list = [] #total_damage_dealt_to_champions_list
tDTaken_list = []  #total_damage_taken_list
tMKilled_list = []  #total_minions_killed_list
turKills_list = []  #turret_kills_list
win_list = []


for i in range(0,2000): 
    pair = match_id_df_6.iloc[i,:].to_list()#='='='='='='='='='='='='='='='==='=='='='='
    puuid = pair[0]
    match_id = pair[1]
    url = f'https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}'

    # Make the Riot API call
    response = requests.get(url)
    response_json = response.json()

    
    kills = None 
    assists = None  # set a default value
    deaths = None 
    champExp = None 
    champName = None 
    indiPos = None 
    teamPos = None 
    lane = None 
    role = None 
    gold = None 
    tDDealt = None 
    tDDealtChamp = None 
    tDTaken = None 
    tMKilled = None
    turKills = None
    win = None
    for participant in response_json['info']['participants']:
      if participant['puuid'] == puuid and participant['teamPosition'] == 'MIDDLE': #게임에 참여한 참가자들 중 위에서 우리가 구한 puuid와 puuid 가 일치하고 individualPosition이 MIDDLE인 participants에 대해서만 정보 추출 
        kills = participant['kills']
        assists = participant['assists']
        deaths = participant["deaths"]
        champExp = participant["champExperience"]
        champName = participant["championName"]
        indiPos = participant["individualPosition"]
        teamPos = participant["teamPosition"]
        lane = participant["lane"]
        role = participant["role"]
        gold = participant["goldEarned"]
        tDDealt = participant["totalDamageDealt"]
        tDDealtChamp = participant["totalDamageDealtToChampions"]
        tDTaken = participant["totalDamageTaken"]
        tMKilled = participant["totalMinionsKilled"]
        turKills = participant["turretKills"]
        win = participant["win"]
        break
    assists_list.append(assists)  # append assists only if it was set to a non-default value : 근데 이렇게 하면 결측치는 그냥 삭제되어버림. 고쳐야 할듯?
    kills_list.append(kills)
    deaths_list.append(deaths)
    champExp_list.append(champExp)
    champName_list.append(champName)
    indiPos_list.append(indiPos)
    teamPos_list.append(teamPos)
    lane_list.append(lane)
    role_list.append(role)
    gold_list.append(gold)
    tDDealt_list.append(tDDealt)
    tDDealtChamp_list.append(tDDealtChamp)
    tDTaken_list.append(tDTaken)
    tMKilled_list.append(tMKilled)
    turKills_list.append(turKills)
    win_list.append(win)

    time.sleep(1)


In [ ]:
#길이
print(len(assists_list))
print(len(kills_list))
print(len(deaths_list))
print(len(champExp_list))
print(len(champName_list))
print(len(indiPos_list))
print(len(teamPos_list))
print(len(lane_list))
print(len(role_list))
print(len(gold_list))
print(len(tDDealt_list))
print(len(tDDealtChamp_list))
print(len(tDTaken_list))
print(len(tMKilled_list))
print(len(turKills_list))
print(len(win_list))

#내용
print(assists_list)
print(kills_list)
print(deaths_list)
print(champExp_list)
print(champName_list)
print(indiPos_list)
print(teamPos_list)
print(lane_list)
print(role_list)
print(gold_list)
print(tDDealt_list)
print(tDDealtChamp_list)
print(tDTaken_list)
print(tMKilled_list)
print(turKills_list)
print(win_list)

In [ ]:
matchdata_6 = pd.DataFrame()
matchdata_6['assists'] = assists_list
matchdata_6['kills'] = kills_list
matchdata_6['deaths'] = deaths_list
matchdata_6['champExp'] = champExp_list #champ_experience_list
matchdata_6['champName'] = champName_list #champion_name_list
matchdata_6['indiPos'] = indiPos_list #individual_position_list
matchdata_6['lane'] = lane_list
matchdata_6['role'] = role_list 
matchdata_6['gold'] = gold_list    #gold_earned_list
matchdata_6['teamPos'] = teamPos_list 
matchdata_6['tDDealt'] = tDDealt_list  #total_damage_dealt_list
matchdata_6['tDDealtChamp'] = tDDealtChamp_list #total_damage_dealt_to_champions_list
matchdata_6['tDTaken'] = tDTaken_list  #total_damage_taken_list
matchdata_6['tMKilled'] = tMKilled_list   #total_minions_killed_list
matchdata_6['turKills'] = turKills_list   #turret_kills_list
matchdata_6['win'] = win_list 

print(matchdata_6.info())
print(matchdata_6.head(20))

In [50]:
matchdata_6.to_csv('matchdata_6(0326).csv', header=True, index=False)

### **MATCH 7**

In [5]:
region = 'asia'
# Empty lists to store the collected data
# game_mode_list = [] 얘넨 나중에
# game_type_list = []

assists_list = []
kills_list = []
deaths_list = []
champExp_list = [] #champ_experience_list
champName_list = [] #champion_name_list
indiPos_list = [] #individual_position_list
lane_list = []
role_list = []
gold_list = []   #gold_earned_list
teamPos_list = []
tDDealt_list = []  #total_damage_dealt_list
tDDealtChamp_list = [] #total_damage_dealt_to_champions_list
tDTaken_list = []  #total_damage_taken_list
tMKilled_list = []  #total_minions_killed_list
turKills_list = []  #turret_kills_list
win_list = []


for i in range(0,2000): #########길이 수정 요망########
    pair = match_id_df_7.iloc[i,:].to_list() #='='='='='='='='='='='='='='='==='=='='='='
    puuid = pair[0]
    match_id = pair[1]
    url = f'https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}'

    # Make the Riot API call
    response = requests.get(url)
    response_json = response.json()

    
    kills = None 
    assists = None  # set a default value
    deaths = None 
    champExp = None 
    champName = None 
    indiPos = None 
    teamPos = None 
    lane = None 
    role = None 
    gold = None 
    tDDealt = None 
    tDDealtChamp = None 
    tDTaken = None 
    tMKilled = None
    turKills = None
    win = None
    for participant in response_json['info']['participants']:
      if participant['puuid'] == puuid and participant['teamPosition'] == 'MIDDLE': #게임에 참여한 참가자들 중 위에서 우리가 구한 puuid와 puuid 가 일치하고 individualPosition이 MIDDLE인 participants에 대해서만 정보 추출 
        kills = participant['kills']
        assists = participant['assists']
        deaths = participant["deaths"]
        champExp = participant["champExperience"]
        champName = participant["championName"]
        indiPos = participant["individualPosition"]
        teamPos = participant["teamPosition"]
        lane = participant["lane"]
        role = participant["role"]
        gold = participant["goldEarned"]
        tDDealt = participant["totalDamageDealt"]
        tDDealtChamp = participant["totalDamageDealtToChampions"]
        tDTaken = participant["totalDamageTaken"]
        tMKilled = participant["totalMinionsKilled"]
        turKills = participant["turretKills"]
        win = participant["win"]
        break
    assists_list.append(assists)  # append assists only if it was set to a non-default value : 근데 이렇게 하면 결측치는 그냥 삭제되어버림. 고쳐야 할듯?
    kills_list.append(kills)
    deaths_list.append(deaths)
    champExp_list.append(champExp)
    champName_list.append(champName)
    indiPos_list.append(indiPos)
    teamPos_list.append(teamPos)
    lane_list.append(lane)
    role_list.append(role)
    gold_list.append(gold)
    tDDealt_list.append(tDDealt)
    tDDealtChamp_list.append(tDDealtChamp)
    tDTaken_list.append(tDTaken)
    tMKilled_list.append(tMKilled)
    turKills_list.append(turKills)
    win_list.append(win)

    time.sleep(1)

In [ ]:
#길이
print(len(assists_list))
print(len(kills_list))
print(len(deaths_list))
print(len(champExp_list))
print(len(champName_list))
print(len(indiPos_list))
print(len(teamPos_list))
print(len(lane_list))
print(len(role_list))
print(len(gold_list))
print(len(tDDealt_list))
print(len(tDDealtChamp_list))
print(len(tDTaken_list))
print(len(tMKilled_list))
print(len(turKills_list))
print(len(win_list))

#내용
print(assists_list)
print(kills_list)
print(deaths_list)
print(champExp_list)
print(champName_list)
print(indiPos_list)
print(teamPos_list)
print(lane_list)
print(role_list)
print(gold_list)
print(tDDealt_list)
print(tDDealtChamp_list)
print(tDTaken_list)
print(tMKilled_list)
print(turKills_list)
print(win_list)

In [ ]:
matchdata_7 = pd.DataFrame()
matchdata_7['assists'] = assists_list
matchdata_7['kills'] = kills_list
matchdata_7['deaths'] = deaths_list
matchdata_7['champExp'] = champExp_list #champ_experience_list
matchdata_7['champName'] = champName_list #champion_name_list
matchdata_7['indiPos'] = indiPos_list #individual_position_list
matchdata_7['lane'] = lane_list
matchdata_7['role'] = role_list 
matchdata_7['gold'] = gold_list    #gold_earned_list
matchdata_7['teamPos'] = teamPos_list 
matchdata_7['tDDealt'] = tDDealt_list  #total_damage_dealt_list
matchdata_7['tDDealtChamp'] = tDDealtChamp_list #total_damage_dealt_to_champions_list
matchdata_7['tDTaken'] = tDTaken_list  #total_damage_taken_list
matchdata_7['tMKilled'] = tMKilled_list   #total_minions_killed_list
matchdata_7['turKills'] = turKills_list   #turret_kills_list
matchdata_7['win'] = win_list 

print(matchdata_7.info())
print(matchdata_7.head(20))

In [8]:
matchdata_7.to_csv('matchdata_7(0326).csv', header=True, index=False)

### **MATCH 8**

In [6]:
region = 'asia'
# Empty lists to store the collected data
# game_mode_list = [] 얘넨 나중에
# game_type_list = []

assists_list = []
kills_list = []
deaths_list = []
champExp_list = [] #champ_experience_list
champName_list = [] #champion_name_list
indiPos_list = [] #individual_position_list
lane_list = []
role_list = []
gold_list = []   #gold_earned_list
teamPos_list = []
tDDealt_list = []  #total_damage_dealt_list
tDDealtChamp_list = [] #total_damage_dealt_to_champions_list
tDTaken_list = []  #total_damage_taken_list
tMKilled_list = []  #total_minions_killed_list
turKills_list = []  #turret_kills_list
win_list = []


for i in range(0,2000): #########길이 수정 요망########
    pair = match_id_df_8.iloc[i,:].to_list() #='='='='='='='='='='='='='='='==='=='='='='
    puuid = pair[0]
    match_id = pair[1]
    url = f'https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}'

    # Make the Riot API call
    response = requests.get(url)
    response_json = response.json()

    
    kills = None 
    assists = None  # set a default value
    deaths = None 
    champExp = None 
    champName = None 
    indiPos = None 
    teamPos = None 
    lane = None 
    role = None 
    gold = None 
    tDDealt = None 
    tDDealtChamp = None 
    tDTaken = None 
    tMKilled = None
    turKills = None
    win = None
    for participant in response_json['info']['participants']:
      if participant['puuid'] == puuid and participant['teamPosition'] == 'MIDDLE': #게임에 참여한 참가자들 중 위에서 우리가 구한 puuid와 puuid 가 일치하고 individualPosition이 MIDDLE인 participants에 대해서만 정보 추출 
        kills = participant['kills']
        assists = participant['assists']
        deaths = participant["deaths"]
        champExp = participant["champExperience"]
        champName = participant["championName"]
        indiPos = participant["individualPosition"]
        teamPos = participant["teamPosition"]
        lane = participant["lane"]
        role = participant["role"]
        gold = participant["goldEarned"]
        tDDealt = participant["totalDamageDealt"]
        tDDealtChamp = participant["totalDamageDealtToChampions"]
        tDTaken = participant["totalDamageTaken"]
        tMKilled = participant["totalMinionsKilled"]
        turKills = participant["turretKills"]
        win = participant["win"]
        break
    assists_list.append(assists)  # append assists only if it was set to a non-default value : 근데 이렇게 하면 결측치는 그냥 삭제되어버림. 고쳐야 할듯?
    kills_list.append(kills)
    deaths_list.append(deaths)
    champExp_list.append(champExp)
    champName_list.append(champName)
    indiPos_list.append(indiPos)
    teamPos_list.append(teamPos)
    lane_list.append(lane)
    role_list.append(role)
    gold_list.append(gold)
    tDDealt_list.append(tDDealt)
    tDDealtChamp_list.append(tDDealtChamp)
    tDTaken_list.append(tDTaken)
    tMKilled_list.append(tMKilled)
    turKills_list.append(turKills)
    win_list.append(win)

    time.sleep(1)

In [ ]:
#길이
print(len(assists_list))
print(len(kills_list))
print(len(deaths_list))
print(len(champExp_list))
print(len(champName_list))
print(len(indiPos_list))
print(len(teamPos_list))
print(len(lane_list))
print(len(role_list))
print(len(gold_list))
print(len(tDDealt_list))
print(len(tDDealtChamp_list))
print(len(tDTaken_list))
print(len(tMKilled_list))
print(len(turKills_list))
print(len(win_list))

#내용
print(assists_list)
print(kills_list)
print(deaths_list)
print(champExp_list)
print(champName_list)
print(indiPos_list)
print(teamPos_list)
print(lane_list)
print(role_list)
print(gold_list)
print(tDDealt_list)
print(tDDealtChamp_list)
print(tDTaken_list)
print(tMKilled_list)
print(turKills_list)
print(win_list)

In [ ]:
matchdata_8= pd.DataFrame()
matchdata_8['assists'] = assists_list
matchdata_8['kills'] = kills_list
matchdata_8['deaths'] = deaths_list
matchdata_8['champExp'] = champExp_list #champ_experience_list
matchdata_8['champName'] = champName_list #champion_name_list
matchdata_8['indiPos'] = indiPos_list #individual_position_list
matchdata_8['lane'] = lane_list
matchdata_8['role'] = role_list 
matchdata_8['gold'] = gold_list    #gold_earned_list
matchdata_8['teamPos'] = teamPos_list 
matchdata_8['tDDealt'] = tDDealt_list  #total_damage_dealt_list
matchdata_8['tDDealtChamp'] = tDDealtChamp_list #total_damage_dealt_to_champions_list
matchdata_8['tDTaken'] = tDTaken_list  #total_damage_taken_list
matchdata_8['tMKilled'] = tMKilled_list   #total_minions_killed_list
matchdata_8['turKills'] = turKills_list   #turret_kills_list
matchdata_8['win'] = win_list 

print(matchdata_8.info())
print(matchdata_8.head(20))

In [10]:
matchdata_8.to_csv('matchdata_8(0326).csv', header=True, index=False)

### **MATCH 9**

In [5]:
region = 'asia'
# Empty lists to store the collected data
# game_mode_list = [] 얘넨 나중에
# game_type_list = []

assists_list = []
kills_list = []
deaths_list = []
champExp_list = [] #champ_experience_list
champName_list = [] #champion_name_list
indiPos_list = [] #individual_position_list
lane_list = []
role_list = []
gold_list = []   #gold_earned_list
teamPos_list = []
tDDealt_list = []  #total_damage_dealt_list
tDDealtChamp_list = [] #total_damage_dealt_to_champions_list
tDTaken_list = []  #total_damage_taken_list
tMKilled_list = []  #total_minions_killed_list
turKills_list = []  #turret_kills_list
win_list = []


for i in range(0,2000): #########길이 수정 요망########
    pair = match_id_df_9.iloc[i,:].to_list() #='='='='='='='='='='='='='='='==='=='='='='
    puuid = pair[0]
    match_id = pair[1]
    url = f'https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}'

    # Make the Riot API call
    response = requests.get(url)
    response_json = response.json()

    
    kills = None 
    assists = None  # set a default value
    deaths = None 
    champExp = None 
    champName = None 
    indiPos = None 
    teamPos = None 
    lane = None 
    role = None 
    gold = None 
    tDDealt = None 
    tDDealtChamp = None 
    tDTaken = None 
    tMKilled = None
    turKills = None
    win = None
    for participant in response_json['info']['participants']:
      if participant['puuid'] == puuid and participant['teamPosition'] == 'MIDDLE': #게임에 참여한 참가자들 중 위에서 우리가 구한 puuid와 puuid 가 일치하고 individualPosition이 MIDDLE인 participants에 대해서만 정보 추출 
        kills = participant['kills']
        assists = participant['assists']
        deaths = participant["deaths"]
        champExp = participant["champExperience"]
        champName = participant["championName"]
        indiPos = participant["individualPosition"]
        teamPos = participant["teamPosition"]
        lane = participant["lane"]
        role = participant["role"]
        gold = participant["goldEarned"]
        tDDealt = participant["totalDamageDealt"]
        tDDealtChamp = participant["totalDamageDealtToChampions"]
        tDTaken = participant["totalDamageTaken"]
        tMKilled = participant["totalMinionsKilled"]
        turKills = participant["turretKills"]
        win = participant["win"]
        break
    assists_list.append(assists)  # append assists only if it was set to a non-default value : 근데 이렇게 하면 결측치는 그냥 삭제되어버림. 고쳐야 할듯?
    kills_list.append(kills)
    deaths_list.append(deaths)
    champExp_list.append(champExp)
    champName_list.append(champName)
    indiPos_list.append(indiPos)
    teamPos_list.append(teamPos)
    lane_list.append(lane)
    role_list.append(role)
    gold_list.append(gold)
    tDDealt_list.append(tDDealt)
    tDDealtChamp_list.append(tDDealtChamp)
    tDTaken_list.append(tDTaken)
    tMKilled_list.append(tMKilled)
    turKills_list.append(turKills)
    win_list.append(win)

    time.sleep(1)

In [ ]:

#길이
print(len(assists_list))
print(len(kills_list))
print(len(deaths_list))
print(len(champExp_list))
print(len(champName_list))
print(len(indiPos_list))
print(len(teamPos_list))
print(len(lane_list))
print(len(role_list))
print(len(gold_list))
print(len(tDDealt_list))
print(len(tDDealtChamp_list))
print(len(tDTaken_list))
print(len(tMKilled_list))
print(len(turKills_list))
print(len(win_list))

#내용
print(assists_list)
print(kills_list)
print(deaths_list)
print(champExp_list)
print(champName_list)
print(indiPos_list)
print(teamPos_list)
print(lane_list)
print(role_list)
print(gold_list)
print(tDDealt_list)
print(tDDealtChamp_list)
print(tDTaken_list)
print(tMKilled_list)
print(turKills_list)
print(win_list)

In [ ]:
matchdata_9= pd.DataFrame()
matchdata_9['assists'] = assists_list
matchdata_9['kills'] = kills_list
matchdata_9['deaths'] = deaths_list
matchdata_9['champExp'] = champExp_list #champ_experience_list
matchdata_9['champName'] = champName_list #champion_name_list
matchdata_9['indiPos'] = indiPos_list #individual_position_list
matchdata_9['lane'] = lane_list
matchdata_9['role'] = role_list 
matchdata_9['gold'] = gold_list    #gold_earned_list
matchdata_9['teamPos'] = teamPos_list 
matchdata_9['tDDealt'] = tDDealt_list  #total_damage_dealt_list
matchdata_9['tDDealtChamp'] = tDDealtChamp_list #total_damage_dealt_to_champions_list
matchdata_9['tDTaken'] = tDTaken_list  #total_damage_taken_list
matchdata_9['tMKilled'] = tMKilled_list   #total_minions_killed_list
matchdata_9['turKills'] = turKills_list   #turret_kills_list
matchdata_9['win'] = win_list 

print(matchdata_9.info())
print(matchdata_9.head(20))

In [9]:
matchdata_9.to_csv('matchdata_9(0326).csv', header=True, index=False)

### **MATCH 10**

In [11]:
region = 'asia'
# Empty lists to store the collected data
# game_mode_list = [] 얘넨 나중에
# game_type_list = []

assists_list = []
kills_list = []
deaths_list = []
champExp_list = [] #champ_experience_list
champName_list = [] #champion_name_list
indiPos_list = [] #individual_position_list
lane_list = []
role_list = []
gold_list = []   #gold_earned_list
teamPos_list = []
tDDealt_list = []  #total_damage_dealt_list
tDDealtChamp_list = [] #total_damage_dealt_to_champions_list
tDTaken_list = []  #total_damage_taken_list
tMKilled_list = []  #total_minions_killed_list
turKills_list = []  #turret_kills_list
win_list = []


for i in range(0,2000): #########길이 수정 요망########
    pair = match_id_df_10.iloc[i,:].to_list() #='='='='='='='='='='='='='='='==='=='='='='
    puuid = pair[0]
    match_id = pair[1]
    url = f'https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}'

    # Make the Riot API call
    response = requests.get(url)
    response_json = response.json()

    
    kills = None 
    assists = None  # set a default value
    deaths = None 
    champExp = None 
    champName = None 
    indiPos = None 
    teamPos = None 
    lane = None 
    role = None 
    gold = None 
    tDDealt = None 
    tDDealtChamp = None 
    tDTaken = None 
    tMKilled = None
    turKills = None
    win = None
    for participant in response_json['info']['participants']:
      if participant['puuid'] == puuid and participant['teamPosition'] == 'MIDDLE': #게임에 참여한 참가자들 중 위에서 우리가 구한 puuid와 puuid 가 일치하고 individualPosition이 MIDDLE인 participants에 대해서만 정보 추출 
        kills = participant['kills']
        assists = participant['assists']
        deaths = participant["deaths"]
        champExp = participant["champExperience"]
        champName = participant["championName"]
        indiPos = participant["individualPosition"]
        teamPos = participant["teamPosition"]
        lane = participant["lane"]
        role = participant["role"]
        gold = participant["goldEarned"]
        tDDealt = participant["totalDamageDealt"]
        tDDealtChamp = participant["totalDamageDealtToChampions"]
        tDTaken = participant["totalDamageTaken"]
        tMKilled = participant["totalMinionsKilled"]
        turKills = participant["turretKills"]
        win = participant["win"]
        break
    assists_list.append(assists)  # append assists only if it was set to a non-default value : 근데 이렇게 하면 결측치는 그냥 삭제되어버림. 고쳐야 할듯?
    kills_list.append(kills)
    deaths_list.append(deaths)
    champExp_list.append(champExp)
    champName_list.append(champName)
    indiPos_list.append(indiPos)
    teamPos_list.append(teamPos)
    lane_list.append(lane)
    role_list.append(role)
    gold_list.append(gold)
    tDDealt_list.append(tDDealt)
    tDDealtChamp_list.append(tDDealtChamp)
    tDTaken_list.append(tDTaken)
    tMKilled_list.append(tMKilled)
    turKills_list.append(turKills)
    win_list.append(win)

    time.sleep(1)

In [ ]:
#길이
print(len(assists_list))
print(len(kills_list))
print(len(deaths_list))
print(len(champExp_list))
print(len(champName_list))
print(len(indiPos_list))
print(len(teamPos_list))
print(len(lane_list))
print(len(role_list))
print(len(gold_list))
print(len(tDDealt_list))
print(len(tDDealtChamp_list))
print(len(tDTaken_list))
print(len(tMKilled_list))
print(len(turKills_list))
print(len(win_list))

#내용
print(assists_list)
print(kills_list)
print(deaths_list)
print(champExp_list)
print(champName_list)
print(indiPos_list)
print(teamPos_list)
print(lane_list)
print(role_list)
print(gold_list)
print(tDDealt_list)
print(tDDealtChamp_list)
print(tDTaken_list)
print(tMKilled_list)
print(turKills_list)
print(win_list)

In [ ]:
matchdata_10= pd.DataFrame()
matchdata_10['assists'] = assists_list
matchdata_10['kills'] = kills_list
matchdata_10['deaths'] = deaths_list
matchdata_10['champExp'] = champExp_list #champ_experience_list
matchdata_10['champName'] = champName_list #champion_name_list
matchdata_10['indiPos'] = indiPos_list #individual_position_list
matchdata_10['lane'] = lane_list
matchdata_10['role'] = role_list 
matchdata_10['gold'] = gold_list    #gold_earned_list
matchdata_10['teamPos'] = teamPos_list 
matchdata_10['tDDealt'] = tDDealt_list  #total_damage_dealt_list
matchdata_10['tDDealtChamp'] = tDDealtChamp_list #total_damage_dealt_to_champions_list
matchdata_10['tDTaken'] = tDTaken_list  #total_damage_taken_list
matchdata_10['tMKilled'] = tMKilled_list   #total_minions_killed_list
matchdata_10['turKills'] = turKills_list   #turret_kills_list
matchdata_10['win'] = win_list 

print(matchdata_10.info())
print(matchdata_10.head(20))

In [15]:
matchdata_10.to_csv('matchdata_10(0326).csv', header=True, index=False)

# **Match DATA 하나의 데이터프레임으로 통합하기**

In [3]:
# 불러오기 
import pandas as pd
matchdata_1 = pd.read_csv('/content/drive/MyDrive/LOL_ANALYSIS_0323/matchdata_1(0326).csv')
matchdata_2 = pd.read_csv('/content/drive/MyDrive/LOL_ANALYSIS_0323/matchdata_2(0326).csv')
matchdata_3 = pd.read_csv('/content/drive/MyDrive/LOL_ANALYSIS_0323/matchdata_3(0326).csv')
matchdata_4 = pd.read_csv('/content/drive/MyDrive/LOL_ANALYSIS_0323/matchdata_4(0326).csv')
matchdata_5 = pd.read_csv('/content/drive/MyDrive/LOL_ANALYSIS_0323/matchdata_5(0326).csv')
matchdata_6 = pd.read_csv('/content/drive/MyDrive/LOL_ANALYSIS_0323/matchdata_6(0326).csv')
matchdata_7 = pd.read_csv('/content/drive/MyDrive/LOL_ANALYSIS_0323/matchdata_7(0326).csv')
matchdata_8 = pd.read_csv('/content/drive/MyDrive/LOL_ANALYSIS_0323/matchdata_8(0326).csv')
matchdata_9 = pd.read_csv('/content/drive/MyDrive/LOL_ANALYSIS_0323/matchdata_9(0326).csv')
matchdata_10 = pd.read_csv('/content/drive/MyDrive/LOL_ANALYSIS_0323/matchdata_10(0326).csv')


In [4]:
GOLD1 = pd.concat([matchdata_1, matchdata_2, matchdata_3, matchdata_4, matchdata_5, matchdata_6, matchdata_7, matchdata_8, matchdata_9, matchdata_10])
print(GOLD1.info())
print(GOLD1.head(10))
print(GOLD1.tail(10))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 0 to 1999
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   assists       3505 non-null   float64
 1   kills         3505 non-null   float64
 2   deaths        3505 non-null   float64
 3   champExp      3505 non-null   float64
 4   champName     3505 non-null   object 
 5   indiPos       3505 non-null   object 
 6   lane          3505 non-null   object 
 7   role          3505 non-null   object 
 8   gold          3505 non-null   float64
 9   teamPos       3505 non-null   object 
 10  tDDealt       3505 non-null   float64
 11  tDDealtChamp  3505 non-null   float64
 12  tDTaken       3505 non-null   float64
 13  tMKilled      3505 non-null   float64
 14  turKills      3505 non-null   float64
 15  win           3505 non-null   object 
dtypes: float64(10), object(6)
memory usage: 2.6+ MB
None
   assists  kills  deaths  champExp champName indiPos    lan

In [6]:
# 결측치 제거, GOLD제거  생각해보기 
GOLD1 = GOLD1.dropna()

# **Match DATA csv 파일로 저장**

In [7]:
GOLD1.to_csv('GOLD1.csv', header=True, index=False)